<a href="https://colab.research.google.com/github/Deep-Dey1/MISTRAL-RAG/blob/main/Result_analysis_finetuning_vs_rag_based_on_mistral_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Clear GPU & System RAM

In [6]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Run garbage collection
gc.collect()

# Restart runtime (optional, but ensures a fresh start)
!kill -9 -1  # WARNING: This will restart your Colab session.

Step 2: Install Dependencies

In [1]:
!pip install torch transformers datasets peft sentence-transformers faiss-cpu wandb matplotlib scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Step 3: Load 20 Queries from the Alpaca Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("tatsu-lab/alpaca", split="train")

# Select 20 queries
eval_data = dataset.shuffle(seed=42).select(range(20))

# Format input queries
eval_prompts = [
    f"###Human: {ex['instruction']} {ex['input']} ###Assistant:" for ex in eval_data
]

# Ground truth responses
ground_truth_responses = [ex["output"] for ex in eval_data]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Step 4: Load Tokenizer & Finetuned Model

In [4]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `mistral-rag-push` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentica

In [ ]:
import torch
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("deep0210/mistral-finetuned-alpaca")

# Load finetuned model with PEFT (fixing LoRA mismatch)
model_finetuned = AutoPeftModelForCausalLM.from_pretrained(
    "deep0210/mistral-finetuned-alpaca",
    device_map="auto"
)

# Move model to GPU
model_finetuned.to("cuda")


 Step 5: Generate Responses from the Finetuned Model

In [ ]:
def generate_responses(model, tokenizer, prompts):
    model.eval()
    responses = []

    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            output = model.generate(**inputs, max_length=200)

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        responses.append(generated_text)

    return responses

# Get finetuned model responses
finetuned_responses = generate_responses(model_finetuned, tokenizer, eval_prompts)


Step 6: Compute Performance Metrics

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from datasets import load_metric
import numpy as np

# Load evaluation models
bleu = load_metric("sacrebleu")
rouge = load_metric("rouge")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Function to compute metrics
def evaluate_model(generated, reference):
    # Compute BLEU
    bleu_score = bleu.compute(predictions=generated, references=[[r] for r in reference])["score"]

    # Compute ROUGE
    rouge_scores = rouge.compute(predictions=generated, references=reference)

    # Compute Cosine Similarity using embeddings
    gen_embeddings = embedder.encode(generated)
    ref_embeddings = embedder.encode(reference)
    cosine_sim = np.mean([cosine_similarity([g], [r])[0][0] for g, r in zip(gen_embeddings, ref_embeddings)])

    return {
        "BLEU": bleu_score,
        "ROUGE-L": rouge_scores["rougeL"].mid.fmeasure,
        "Cosine Similarity": cosine_sim
    }

# Evaluate finetuned model
finetuned_metrics = evaluate_model(finetuned_responses, ground_truth_responses)
print(f"Finetuned Model Metrics: {finetuned_metrics}")


Step 7: Plot Performance Graphs

In [ ]:
import matplotlib.pyplot as plt

# Plot function
def plot_metrics(metrics, title):
    keys = list(metrics.keys())
    values = list(metrics.values())

    plt.figure(figsize=(8, 5))
    plt.bar(keys, values, color=['blue', 'green', 'red'])
    plt.xlabel("Metric")
    plt.ylabel("Score")
    plt.title(title)
    plt.ylim(0, 1)  # Normalize to 0-1 scale
    plt.show()

# Plot finetuned model metrics
plot_metrics(finetuned_metrics, "Finetuned Mistral Model Performance")


Step 8: Repeat for RAG Model

In [ ]:
# Load RAG model
model_rag = AutoModelForCausalLM.from_pretrained("deep0210/mistral-rag", device_map="auto").to("cuda")

# Generate responses from RAG model
rag_responses = generate_responses(model_rag, tokenizer, eval_prompts)

# Evaluate RAG model
rag_metrics = evaluate_model(rag_responses, ground_truth_responses)
print(f"RAG Model Metrics: {rag_metrics}")

# Plot RAG model performance
plot_metrics(rag_metrics, "RAG Mistral Model Performance")


Step 9: Compare Both Models

In [ ]:
# Compare metrics
labels = list(finetuned_metrics.keys())
finetuned_scores = list(finetuned_metrics.values())
rag_scores = list(rag_metrics.values())

x = np.arange(len(labels))  # Label locations
width = 0.35  # Width of bars

fig, ax = plt.subplots(figsize=(8, 5))
rects1 = ax.bar(x - width/2, finetuned_scores, width, label="Finetuned")
rects2 = ax.bar(x + width/2, rag_scores, width, label="RAG")

ax.set_xlabel("Metric")
ax.set_ylabel("Score")
ax.set_title("Comparison of Finetuned vs RAG Model Performance")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()
